# Lab 5

Welcome to lab 5!  In this session, we'll investigate the random variation that happens when we compute an *estimate* of a fact about a population using only a random sample of that population.

As usual, **run the cell below** to prepare the lab and the automatic tests.

In [2]:
# Run this cell, but please don't change it.

# These lines import the NumPy and datascience modules.
import numpy as np
# This way of importing the datascience module lets you write "Table" instead
# of "datascience.Table".  The "*" means "import everything in the module."
from datascience import *
# We've also added some special utility functions for this lab.
from lab_utils import *

# These lines set up visualizations.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# These lines load the tests.
from client.api.assignment import load_assignment 
lab05 = load_assignment('longlab05.ok')

# 1. Warplanes

Let's work through the number-of-warplanes estimation problem you just saw in lecture.

Recall the setup: The British Royal Air Force (RAF) want to know how many warplanes the Germans have.  They hypothesize that the Germans label their planes with serial numbers, starting from 1 for the first plane, and increasing by 1 for each new plane built.

That means that the number of planes the Germans have is equal to the largest serial number among their planes.  Let's call that number `N`.  So the RAF want to estimate the number `N`.

The RAF make the further assumption that each time the Germans send a plane into combat, they send one of their planes chosen *uniformly at random*, and an RAF observer always reports its serial number.  (That's not really true, and it's not exactly what the RAF assumed, but it makes the math simpler for this example.)

Some planes might be observed several times, or not at all.  So if the Germans have 5 planes and send planes into combat 3 times, the RAF might observe the serial numbers 4, 1, and 4.

We're tasked with estimating `N` using a bunch of these random observations of serial numbers.

We're going to simulate that sampling process and try out a few methods for estimating `N`.  Then we'll check how well each method typically does at estimating `N`.

## 1.1. Simulating warplane observations
Suppose that `N` is actually 120 (though the RAF don't know that) and our observers spied serial numbers 14 times.

Let's write a function to simulate these 14 observations.  

In [4]:
# The biggest serial number.
N = 120
# The number of observations.
num_observations = 14

def simulate_observations():
    return np.random.randint(1, N+1, num_observations)

Our function takes no arguments, since its argument list, "`()`", is empty.  It uses a function from NumPy that produces random integers.  In this case, we're asking for 14 random integers, each between 1 and `N`.  (The last number isn't included, so we use `N+1`.)

As an aside, notice that we've named the numbers 120 and 14, rather than just writing `np.random.randint(1, 121, 14)` in our function.  This is a really good practice, for 2 reasons:

1. If we have to use the same numbers elsewhere, we can just use their names instead of remembering and repeating their values.  (In software engineering, this is called the DRY principle, short for Don't Repeat Yourself.)
2. Our code is better documented, since it uses readable names rather than mysterious numbers.  We can even put comments next to the definitions of `N` and `num_observations` to explain what they are.

**Question 1.1.1.** Now let's simulate one set of observations.  In the cell below, set `observations` to a random sample of 14 serial numbers by calling `simulate_observations` appropriately.

In [5]:
observations = ...
observations

In [6]:
_ = lab05.grade("q111")

To see what's going on (and see another example of an empirical histogram), let's draw two histograms:
1. the probability distribution of a single observation, and
2. the empirical distribution of the numbers in your 14 observations.

Each number between 1 and N is equally likely, so we'll make a table of all the numbers from 1 to `N` and draw a histogram from it.  Then we'll make a table out of `observations` and draw a histogram from that.

In [82]:
def plot_serial_numbers(numbers):
    Table().with_column("serial number", numbers).hist(bins=np.arange(1, N+2))
    plt.ylim(0, .25)

plot_serial_numbers(np.arange(1, N+1))
plot_serial_numbers(observations)

## 1.2. Estimation
Given a sample of serial numbers, there are *many different* ways to estimate `N`.  Let's investigate a few.

##### The biggest serial number we see
A straightforward method is to find the biggest serial number among the serial numbers we observe.

**Question 1.2.1.** Use that method to estimate `N` from `observations`.  Give your estimate the name `max_estimate`.

*Hint:* The function `max` can take a single array as its argument; it will return the largest number in that array.

In [83]:
# Set max_estimate to the biggest serial number among the
# observed serial numbers that you simulated in question 3.1.
max_estimate = max(observations)
max_estimate

In [84]:
_ = lab05.grade('q121')

How well does this work?  Since we know `N` is actually 120, we can answer that for our particular sample:

In [85]:
error = N - max_estimate
print("Our estimate is {} off from N.".format(error))

But maybe we got lucky or unlucky.  If we're deciding whether to use this method for a similar task, we'd like to know how it *typically* performs, not how it happened to perform this time.

To see what typically happens, we just run our simulation many times.  The cell below does that.

In [1]:
num_simulations = 20000
bins = np.arange(N - 60 + 1, N + 60 + 1, 1)

def draw_max_distribution():
    simulations = Table().with_column("observations", repeat(simulate_observations, num_simulations))
    maxes = simulations.apply(max, "observations")
    simulations.with_column("max", maxes).hist("max", bins=bins)

draw_max_distribution()

This histogram says how often we get different estimates of `N` using this technique.  It's a histogram displaying the *probability distribution* of this estimate.  Sometimes that's called, somewhat confusingly, the *sampling distribution* of the estimate.

The first thing to notice is that we don't always get the right answer.  You might say that we usually get close to the right answer, but that depends on your definition of "close."  When you're trying to figure out how many warplanes the Germans have, you might want to be even more accurate!

You should also find that the sample max is never bigger than `N`, but it's sometimes smaller.  In other words, you only ever *underestimate* `N` using this technique.

**Question 1.2.2.** Try to explain this phenomenon in your own words.  Discuss with your neighbor if you're stuck!

*Write your answer here, replacing this text.  (Double-click this cell to edit it, and click the "run cell" button to switch back to display mode.)*

##### Extrapolating from the average
Here's another idea that comes from looking at the probability histogram of the serial numbers.

Since the serial numbers are evenly distributed from 1 to `N`, the average of all the serial numbers is roughly in the middle: $\frac{\texttt{N}}{2}$.  Further, the law of averages says (heuristically) that the average of a sample is likely to be close to the average of the population it was sampled from.  So if we multiply the average of our observations by 2, we might get something close to `N`.

If you prefer symbols, here's another way to say that:

$$\texttt{average}(\texttt{all serial numbers}) \approx \frac{\texttt{N}}{2} \\
  \overset{\text{by law of averages}}{\implies} \texttt{average}(\texttt{observations}) \approx \frac{\texttt{N}}{2} \\
  \implies 2 \times \texttt{average}(\texttt{observations}) \approx \texttt{N}$$

Here's a function that computes twice the mean of an array of serial numbers:

In [87]:
def mean_based_estimator(nums):
    return 2*np.average(nums)

**Question 1.2.3.** Use that function to estimate `N` from `observations`.  Call the result `mean_based_estimate`.

In [88]:
# Compute mean_based_estimate using your simulated observations.
mean_based_estimate = ...
mean_based_estimate

In [89]:
_ = lab05.grade('q123')

Again, it's not clear whether this estimate was about as accurate we'd expect from this method, or just a fluke.  As before, we can see how this estimator works by simulating.

In [90]:
def draw_twice_mean_distribution():
    simulations = Table().with_column("observations", repeat(simulate_observations, num_simulations))
    twice_means = simulations.apply(mean_based_estimator, "observations")
    simulations.with_column("twice means", twice_means).hist("twice means", bins=bins)

draw_twice_mean_distribution()

**Question 1.2.4.** Among the two estimators we've seen so far, which would you prefer?  Think about the criteria you would use to decide this.

*Write your answer here, replacing this text.*

Notice something about the histograms we've seen so far:

1. The twice-the-mean estimator has (roughly) a Normal distribution.
2. The max estimator doesn't have a Normal distribution.  Classical statistical techniques, which assume sampling distributions are roughly Normal, wouldn't work very well to help us understand how that estimator works.

##### Something more clever
So far, it looks like our choice is between an estimator that chronically underestimates our target and one that tends to make large errors.  Let's take a look at the distribution of maxes again:

In [91]:
draw_max_distribution()

A good question to ask about this histogram is: "Why can't we just shift the estimates over a bit to get more of them close to 120?"

Another way of thinking about this is that it's unlikely we actually get the biggest serial number in our sample, so it makes sense to guess that the real `N` is something a bit higher than the biggest serial number we observe.  For example, if we see 110, it's probably safe to guess that `N` is 111.

Well, we can act on that.  Let's try a few different "shifted" estimators -- the max of the sample, plus a bit.  First we'll define some functions to make that easier.  (If you can't follow these, that's okay; there will be a lab session later where you can learn how to use functions like this.)

In [92]:
def max_plus_n(n):
    def max_func(nums):
        return max(nums) + n
    return max_func

def simulate_estimates(estimator):
    simulations = Table().with_column("observations", repeat(simulate_observations, num_simulations))
    return simulations.apply(estimator, "observations")

def draw_sampling_distribution(estimator):
    Table().with_column("estimates", simulate_estimates(estimator)).hist("estimates", bins=bins)

draw_sampling_distribution(max_plus_n(5))
draw_sampling_distribution(max_plus_n(10))
draw_sampling_distribution(max_plus_n(15))

In the first two, more of the weight of the bars is near 120.  That means we're typically getting closer to 120 this way, which is what we want.  15 goes too far.  So it looks like shifting over by 5 or 10 is a good idea.

Why?  Here's an idea pointed out by a student in Data 8 lecture.  Think about what would happen if our samples came out evenly-spaced in the interval 1 to 120.  The space between them would be $\frac{120}{14}$, or roughly 8.5.  They'd look like this:

In [93]:
plot_serial_numbers(np.arange(N/num_observations, N, N/num_observations))

The biggest observation, hence our max estimate, would be around 8.5 less than 120.  So we can correct for this by adding back in $\frac{120}{14}$.

In [96]:
adjusted_estimator = max_plus_n(N/num_observations)
draw_sampling_distribution(adjusted_estimator)

It turns out that this is a pretty good way to estimate the number of warplanes.

It's worth noting again that the sampling distribution doesn't look very Normal.  Classical statistical methods wouldn't help you understand it, but we can simulate it precisely with a few lines of code.